In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd
from pandas.core.frame import DataFrame

In [2]:
farm=pd.read_csv('data/farm.csv')
shop=pd.read_csv('data/shop.csv')

In [7]:
def stod(slat,slng,dlat,dlng):#输入：起点纬度、起点经度、终点纬度、终点经度
    url ="http://api.map.baidu.com/direction/v2/transit?" #API地址
    ak = 'cGCdY6KHFma1t35LhDUZ13WI5qG1ni24' #秘钥
    tac_metro = r'&tactics_incity=5'
    real_url = url +"origin="+slat+","+slng+"&destination="+dlat+","+dlng+tac_metro+"&ak="+ak #完整的请求代码
    req = requests.get(real_url)
    t = req.text
    data = json.loads(t) #将数据保存在数组data中
    try:#防止某几条数据报错导致请求终止
        total_duration = data['result']["routes"][0]["duration"]/60
        stepstr = "Total time:"+str(total_duration) #获取该条数据总行程时间
        steps = data['result']["routes"][0]['steps'] #获取该条路径的所有步骤
        for step in steps:
            step_tructions = step[0]["instructions"]#每一步的介绍包括乘坐公交车或地铁线路名
            step_duration = step[0]["duration"]#每一步所花费的时间
            stepstr = stepstr+"/"+step_tructions+":"+str(step_duration/60)
    except:
        stepstr = None
    req.close()
    return stepstr #返回数据为总时长/第一步/第一步耗时/第二步/第二步耗时/...

In [4]:
def ave(lst):
    return sum(lst) / len(lst)

In [100]:
shop['name']

0             华润万家(龙华店)
1             华润万家(益田店)
2             华润万家(沙井店)
3             华润万家(横岗店)
4             华润万家(宝安店)
            ...        
76           华润万家龙岗配送中心
77        华润万家便利超市(翠枫店)
78       华润万家便利超市(支六路店)
79            华润万家(麓园店)
80    华润万家便利超市(二十四区六巷店)
Name: name, Length: 81, dtype: object

In [8]:
only_bus=[]
only_walk=[]
wrong_data=[]
metro=[]
walk=[]
other=[]
traffic=[]
waiting=[]
total=[]
transfer=[]

for o in range (len(farm['lat'])):
    
    d_metro_time_total=[]
    d_walk_time_total=[]
    d_other_time_total=[]
    d_total_time=[]
    d_waiting_time=[]
    d_status=[]
    d_transfer=[]
    d_traffic_time=[]
    
    for d in range(len(shop['lat'])):
        s=str(stod(str(farm['lat'][o]),str(farm['lng'][o]),str(shop['lat'][d]),str(shop['lng'][d])))
        
        if s.find('乘地铁')==-1:
            if s.find('乘')!=-1:
                result='Only bus can arrive'
                i_walk_time_total=-10000
                i_other_time_total=10000
                
            elif s.find('乘')==-1 and s.find('步行')!=-1:
                result='Within walking distance'
                i_walk_time_total=10000
                i_other_time_total=-10000
                
            else:
                result='Wrong cause without walking data, bus data or metro data'
                i_walk_time_total=0
                i_other_time_total=0
            i_metro_time_total=0
            i_total_time=0
            i_waiting_time=0
            i_transfer=-10000
            i_traffic_time=0
        
        elif s.find('乘地铁')!=-1:
            result='Can be arrived by metro'
            i_transfer=s.count('站内换乘')
            first_split = s.split("/")
            dic= {'step':first_split}
            table = DataFrame(dic)
            table[['step','time']] = table['step'].str.split(':', expand=True)
            # table['time'] = table['time'].fillna(0)
    
            i_metro_time=[]
            i_walk_time=[]
            i_other_time=[]
            for i in range(len(table)):
                if table['step'][i].find('乘地铁')!=-1 or table['step'][i].find('换乘')!=-1:
                    i_metro_time.append(float(table['time'][i]))
                elif table['step'][i].find('乘地铁')==-1 and table['step'][i].find('步行')!=-1:
                    i_walk_time.append(float(table['time'][i]))
                elif table['step'][i].find('Total time')!=-1:
                    i_total_time=float(table['time'][i])
                else:
                    i_other_time.append(float(table['time'][i]))
    
            i_metro_time_total=sum(i_metro_time)
            i_walk_time_total=sum(i_walk_time)
            i_other_time_total=sum(i_other_time)
            i_traffic_time=i_metro_time_total+i_walk_time_total+i_other_time_total
            i_waiting_time=i_total_time-i_traffic_time
    
        else:
            result='Wrong cause contradict result'
            i_metro_time_total=404
            i_walk_time_total=404
            i_other_time_total=404
            i_total_time=404
            i_waiting_time=404
            i_transfer=404
            i_traffic_time=404
        
        d_metro_time_total.append(i_metro_time_total)
        d_walk_time_total.append(i_walk_time_total)
        d_other_time_total.append(i_other_time_total)
        d_total_time.append(i_total_time)
        d_waiting_time.append(i_waiting_time)
        d_status.append(result)
        d_transfer.append(i_transfer)
        d_traffic_time.append(i_traffic_time)
        
    d_dic={'Destination':shop['name'],
           'Status':d_status,
           'Metro_time':d_metro_time_total,
           'Walk_time':d_walk_time_total,
           'Other_time':d_other_time_total,
           'Traffic_time':d_traffic_time,
           'Waiting_time':d_waiting_time,
           'Total_time':d_total_time,
           'Transfer':d_transfer}
    d_final=DataFrame(d_dic)
    
    f_only_bus=[]
    f_only_walk=[]
    f_wrong_data=[]
    f_metro=[]
    f_walk=[]
    f_other=[]
    f_traffic=[]
    f_waiting=[]
    f_total=[]
    f_transfer=[]
    for f in range (len(d_final['Destination'])):
        if d_final['Status'][f]=='Only bus can arrive':
            f_only_bus.append(d_final['Other_time'][f])
        elif d_final['Status'][f]=='Within walking distance':
            f_only_walk.append(d_final['Walk_time'][f])
        elif d_final['Status'][f]=='Can be arrived by metro':
            f_metro.append(d_final['Metro_time'][f])
            f_walk.append(d_final['Walk_time'][f])
            f_other.append(d_final['Other_time'][f])
            f_traffic.append(d_final['Traffic_time'][f])
            f_waiting.append(d_final['Waiting_time'][f])
            f_total.append(d_final['Total_time'][f])
            f_transfer.append(d_final['Transfer'][f])
        else:
            f_wrong_data.append(d_final['Transfer'][f])
    
    only_bus.append(len(f_only_bus))
    only_walk.append(len(f_only_walk))
    wrong_data.append(len(f_wrong_data))
    metro.append(ave(f_metro))
    walk.append(ave(f_walk))
    other.append(ave(f_other))
    traffic.append(ave(f_traffic))
    waiting.append(ave(f_waiting))
    total.append(ave(f_total))
    transfer.append(ave(f_transfer))

dic={'Origin':farm['name'],
     'Average_metro_time':metro,
     'Average_walk_time':walk,
     'Average_other_time':other,
     'Average_traffic_time':traffic,
     'Average_waiting_time':waiting,
     'Average_total_time':d_total_time,
     'Average_metro_line_transfer':total,
    'Count_of_only_bus_round':only_bus,
    'Count_of only_walk_round':only_walk,
    'Count_of_wrong_data':wrong_data}
#final=DataFrame(dic)
#final
dic

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

reference

https://www.cnblogs.com/jqpy1994/p/10842795.html

https://www.pythonf.cn/read/45820